# Classify

In [1]:
from ipynb.fs.full.koselleck import *
from lltk.model.classifier import *

In [2]:
# get_all_novelty_scores??

In [3]:
def class_word(row):
    if not row.is_signif: return 'Pivots_Never'
#     return f'Pivots_{periodize_bystep(row.changepoint,20,1720,1900)}'
    return f'Pivots_{periodize_sattelzeit(row.changepoint)}'

def class_words(df=None):
    df=get_all_novelty_scores() if df is None else df
    o=[]
    for word,wdf in tqdm(df.groupby('word')):
        wdf=wdf.sort_values('period')
        wdf_nov=wdf[wdf.is_signif==True]
        wdfm=wdf.mean()
        wdfm_nov=wdf_nov.mean()
        changepoint=wdf_nov.iloc[0].period if len(wdf_nov) else np.nan
        o+=[dict(
            word=word,
            changepoint=changepoint,
            is_signif=bool(len(wdf_nov)),
            num_signif=len(wdf_nov),
            foote_novelty=wdfm.foote_novelty,
            foote_novelty_z=wdfm.foote_novelty_z,
            foote_novelty_signif=wdfm_nov.foote_novelty,
            foote_novelty_z_signif=wdfm_nov.foote_novelty_z,
        )]
    odf=pd.DataFrame(o)
    odf['cls']=odf.apply(class_word,axis=1)
    return odf.set_index('word')

In [4]:
dfclasses=class_words()
dfclasses

100%|██████████| 5712/5712 [00:15<00:00, 376.24it/s]


,changepoint,is_signif,num_signif,foote_novelty,foote_novelty_z,foote_novelty_signif,foote_novelty_z_signif,cls
word,,,,,,,,
abbe,1750.0,True,2,799.697494,0.438395,2215.168539,3.094666,Pivots_1700-1770
abbey,NaN,False,0,951.874081,0.723970,NaN,NaN,Pivots_Never
abbot,1815.0,True,2,826.094958,0.487933,1718.181818,2.162021,Pivots_1770-1830
abhorrence,NaN,False,0,655.225806,0.167280,NaN,NaN,Pivots_Never
abilities,1805.0,True,1,865.989972,0.562800,1461.049285,1.679487,Pivots_1770-1830
...,...,...,...,...,...,...,...,...
youth,NaN,False,0,599.278464,0.062289,NaN,NaN,Pivots_Never
youthful,1815.0,True,3,829.906412,0.495085,1453.264009,1.664877,Pivots_1770-1830
youths,1810.0,True,2,676.007708,0.206279,2308.559499,3.269923,Pivots_1770-1830


## Classify by ranks

In [5]:
dfranks=get_all_word_rankings()
dfranks=dfranks[[c for c in dfranks.columns if 'Freq' not in c]]
dfranks

,+Abs,+Conc,+Active,+Passive,+Ambig,-Ambig,+Collective,+Indiv,+Human,+Object,...,+Plural,+Strong,+Weak,+Time,+Space,+Virtue,+Vice,+Woman,+Man,+LNM
word,,,,,,,,,,,,,,,,,,,,,
lack,1,9976,9906,71,7336,2640,665,9312,607,9370,...,NaN,539,9438,6683,3294,4,9973,9252,725,3.0
lax,2,9975,6988,2989,5894,4083,6059,3918,15,9962,...,NaN,744,9233,110,9867,269,9708,9623,354,NaN
tact,3,9974,9735,242,9245,732,5945,4032,27,9950,...,NaN,10,9967,4113,5864,2,9975,8317,1660,NaN
lucid,4,9973,1460,8517,3606,6371,2533,7444,278,9699,...,NaN,14,9963,82,9895,114,9863,9876,101,104.0
beset,5,9972,9343,634,5996,3981,549,9428,122,9855,...,NaN,3092,6885,1966,8011,1486,8491,8613,1364,682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
conduit,9972,5,619,9358,6519,3458,686,9291,9968,9,...,NaN,9392,585,9895,82,9539,438,133,9844,11.0
hewn,9973,4,121,9856,6999,2978,8940,1037,9950,27,...,NaN,9939,38,9620,357,9974,3,1613,8364,34.0
slate,9974,3,7,9970,9789,188,9293,684,9951,26,...,45.0,9397,580,9943,34,9907,70,6412,3565,6.0


In [6]:
dfcr = dfranks.join(dfclasses[['cls']])
dfcr = dfcr[~dfcr['cls'].isna()]
lkw=set(get_keywords())
dfcr['is_kw']=['Keyword' if w in lkw else 'Not_Keyword' for w in dfcr.index]
#dfcr['is_sword']=[int('f' in w or 's' in w) for w in dfcr.index]
dfcr=dfcr.dropna(axis=1)
dfcr

,+Abs,+Conc,+Active,+Passive,+Ambig,-Ambig,+Collective,+Indiv,+Human,+Object,...,+Strong,+Weak,+Time,+Space,+Virtue,+Vice,+Woman,+Man,cls,is_kw
word,,,,,,,,,,,,,,,,,,,,,
lack,1,9976,9906,71,7336,2640,665,9312,607,9370,...,539,9438,6683,3294,4,9973,9252,725,Pivots_1830-1900,Not_Keyword
lucid,4,9973,1460,8517,3606,6371,2533,7444,278,9699,...,14,9963,82,9895,114,9863,9876,101,Pivots_1830-1900,Not_Keyword
frenchmen,7,9970,9712,265,9447,530,391,9586,507,9470,...,742,9235,2525,7452,557,9420,9178,799,Pivots_Never,Not_Keyword
role,8,9969,8930,1047,8240,1737,7370,2607,5,9972,...,147,9830,10,9967,105,9872,7163,2814,Pivots_Never,Not_Keyword
deals,9,9968,5078,4899,6539,3438,97,9880,1274,8703,...,121,9856,2524,7453,2155,7822,9232,745,Pivots_1830-1900,Not_Keyword
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mud,9963,14,593,9384,86,9891,1353,8624,9007,970,...,8282,1695,9491,486,9877,100,5056,4921,Pivots_1770-1830,Not_Keyword
ledge,9964,13,712,9265,7531,2446,8882,1095,9960,17,...,9516,461,9777,200,9813,164,2790,7187,Pivots_Never,Not_Keyword
peak,9965,12,455,9522,9867,110,720,9257,9969,8,...,131,9846,9858,119,6916,3061,8772,1205,Pivots_1830-1900,Not_Keyword


## Classifier

### Combo

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

from combo.models.classifier_stacking import Stacking
from combo.utils.data import evaluate_print

In [33]:
dfcr['cls2']=['Pivots_Sometime' if w!='Pivots_Never' else w for w in dfcr.cls]
# dfcr

In [34]:
dfcr.is_kw.value_counts()

Not_Keyword    5504
Keyword         109
Name: is_kw, dtype: int64

In [35]:
def classify(dfcr,
              y_col_name,
              posname,
              num_runs=2,
              n=None):
    
    dfcr['_y_']=[int(y==posname) for y in dfcr[y_col_name]]
    groups=dfcr.groupby('_y_')
    minlen=min(len(g) for i,g in groups)
    n = n if n and n>minlen else minlen
    l_dfr=[]
    l_dfc=[]
    
    iter1=tqdm(range(num_runs),desc='Running classifiers')
    for run in iter1:
        dfsmpl=groups.sample(n=n)
        dfsmpl_q = dfsmpl.drop('_y_',1).select_dtypes('number')
        X, y = (dfsmpl_q, dfsmpl._y_)
        # Define data file and read X and y
        random_state = 42
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=0.2,
            random_state=random_state
        )
        # initialize a group of clfs
        classifiers = [
            DecisionTreeClassifier(random_state=random_state),
            LogisticRegression(random_state=random_state),
            KNeighborsClassifier(),
            RandomForestClassifier(random_state=random_state),
            GradientBoostingClassifier(random_state=random_state),
            SVC(probability=True)
        ]
        clf_names = ['DT', 'LR', 'KNN', 'RF', 'GBDT','SVC','STACK']

        clf_stack = Stacking(
            classifiers, n_folds=4, shuffle_data=False,
            keep_original=True, use_proba=True,
            random_state=random_state
        )

        
        
        # evaluate individual classifiers
        lclassifiers = classifiers# + [clf_stack]
        iterr=tqdm(lclassifiers,disable=True)
        for i, clf in enumerate(iterr):
            iter1.set_description(f'Running {clf_names[i]} classifier')
            clf.fit(X_train, y_train)
            y_test_predict = clf.predict(X_test)
            X_test_proba = clf.predict_proba(X_test)
            dfr=pd.DataFrame(
                X_test_proba,
                index=X_test.index,
                columns=['prob_neg','prob_pos']
            ).assign(pred=y_test_predict, run=run,y_col_name=y_col_name,classifier=clf_names[i])
            l_dfr+=[dfr]
    odf=pd.concat(l_dfr) if len(l_dfr) else pd.DataFrame()
    odf=odf.join(dfcr[['_y_']]).rename(dict(_y_='true'), axis=1)
    odf['correct']=(odf['pred']==odf['true']).apply(int)
    odf['pred_str']=odf.pred.apply(lambda x: posname if x else 'Not_'+posname)
    odf['true_str']=odf.true.apply(lambda x: posname if x else 'Not_'+posname)
    
    for cn,dfrg in odf.groupby('classifier'):
        evaluate_print(cn, dfrg.true, dfrg.pred)
    return odf

In [224]:
dfr=classify(dfcr,'is_kw','Keyword',num_runs=10)
dfr

Running SVC classifier: 100%|██████████| 100/100 [00:34<00:00,  2.86it/s]


DT Accuracy:0.5684, ROC:0.5696, F1:0.6161
GBDT Accuracy:0.6223, ROC:0.625, F1:0.6657
KNN Accuracy:0.6232, ROC:0.6047, F1:0.6908
LR Accuracy:0.6341, ROC:0.6357, F1:0.6783
RF Accuracy:0.6361, ROC:0.6445, F1:0.6721
SVC Accuracy:0.6502, ROC:0.6592, F1:0.685


,prob_neg,prob_pos,pred,run,y_col_name,classifier,true,correct,pred_str,true_str
word,,,,,,,,,,
abode,1.000000,0.000000,0,80,is_kw,DT,0,1,Not_Keyword,Not_Keyword
abode,0.610798,0.389202,0,80,is_kw,LR,0,1,Not_Keyword,Not_Keyword
abode,0.200000,0.800000,1,80,is_kw,KNN,0,0,Keyword,Not_Keyword
abode,0.730000,0.270000,0,80,is_kw,RF,0,1,Not_Keyword,Not_Keyword
abode,0.988715,0.011285,0,80,is_kw,GBDT,0,1,Not_Keyword,Not_Keyword
...,...,...,...,...,...,...,...,...,...,...
zealous,0.289213,0.710787,1,10,is_kw,LR,0,0,Keyword,Not_Keyword
zealous,0.400000,0.600000,1,10,is_kw,KNN,0,0,Keyword,Not_Keyword
zealous,0.380000,0.620000,1,10,is_kw,RF,0,0,Keyword,Not_Keyword


In [225]:
dfr['num']=1
dfr.groupby('word').agg(
    dict(
        num='sum', prob_neg='mean', prob_pos='mean',
        pred='mean',run='mean',true='mean',correct='mean'
    )
).sort_values('prob_pos').query('num>6')

,num,prob_neg,prob_pos,pred,run,true,correct
word,,,,,,,
scene,18,0.926815,0.073185,0.000000,51.000000,0,1.000000
coat,12,0.911345,0.088655,0.000000,61.500000,0,1.000000
animal,12,0.887329,0.112671,0.000000,24.000000,0,1.000000
chairs,12,0.880254,0.119746,0.000000,71.000000,0,1.000000
blue,12,0.877695,0.122305,0.000000,64.000000,0,1.000000
...,...,...,...,...,...,...,...
yields,12,0.186681,0.813319,1.000000,54.000000,0,0.000000
government,168,0.185665,0.814335,0.964286,43.857143,1,0.964286
acceptance,18,0.180430,0.819570,1.000000,45.000000,0,0.000000


In [226]:
classify(dfcr, 'cls', 'Pivots_Never', num_runs=10)

Running SVC classifier: 100%|██████████| 100/100 [19:45<00:00, 11.86s/it]


DT Accuracy:0.5077, ROC:0.5077, F1:0.5086
GBDT Accuracy:0.5369, ROC:0.5364, F1:0.5623
KNN Accuracy:0.5133, ROC:0.5132, F1:0.517
LR Accuracy:0.5316, ROC:0.5317, F1:0.5268
RF Accuracy:0.5304, ROC:0.5305, F1:0.5289
SVC Accuracy:0.5407, ROC:0.5405, F1:0.5551


,prob_neg,prob_pos,pred,run,y_col_name,classifier,true,correct,pred_str,true_str
word,,,,,,,,,,
abbe,0.000000,1.000000,1,2,cls,DT,0,0,Pivots_Never,Not_Pivots_Never
abbe,0.394522,0.605478,1,2,cls,LR,0,0,Pivots_Never,Not_Pivots_Never
abbe,0.200000,0.800000,1,2,cls,KNN,0,0,Pivots_Never,Not_Pivots_Never
abbe,0.440000,0.560000,1,2,cls,RF,0,0,Pivots_Never,Not_Pivots_Never
abbe,0.372423,0.627577,1,2,cls,GBDT,0,0,Pivots_Never,Not_Pivots_Never
...,...,...,...,...,...,...,...,...,...,...
zealous,0.527088,0.472912,0,95,cls,LR,1,0,Not_Pivots_Never,Pivots_Never
zealous,0.800000,0.200000,0,95,cls,KNN,1,0,Not_Pivots_Never,Pivots_Never
zealous,0.650000,0.350000,0,95,cls,RF,1,0,Not_Pivots_Never,Pivots_Never


In [ ]:
classify(dfcr, 'cls', 'Pivots_1770-1830', num_runs=10)

Running RF classifier:  47%|████▋     | 47/100 [04:14<04:47,  5.43s/it]  

In [ ]:
classify(dfcr, 'cls', 'Pivots_1700-1770', num_runs=10)

In [ ]:
classify(dfcr, 'cls', 'Pivots_1830-1900', num_runs=10)

## By group

In [21]:
# def classify_runs(dfcr,y_col_name='cls',num_runs=100,n=500):
#     l_dfc =[]
#     l_dfr =[]
#     for cls,dfcls in tqdm(dfcr.groupby(y_col_name),position=0):
#         dfcls_not = dfcr[dfcr[y_col_name]!=cls]
#         if str(cls).startswith('Not_'): continue
#         idf=dfcls.append(dfcls_not.assign(**{y_col_name:'Not_'+str(cls)}))
#         for run in tqdm(range(num_runs),position=):
#             idf_smpl=idf.groupby(y_col_name).sample(n=n)
#             clf = Classifier(idf)
#             clf.classify(y_col_name,standardize=True,resample=False)        
#             l_dfc.append(clf.dfc.assign(cls=cls, run=run))
#             l_dfr.append(clf.dfr.assign(cls=cls, run=run))
#     dfcdf = pd.concat(l_dfc)
#     dfrdf = pd.concat(l_dfr)
#     return dfrdf,dfcdf

In [22]:
# dfcr

In [23]:
def show_results(dfrdf,dfcdf):
    display(dfrdf.groupby('cls').mean().sort_values('correct',ascending=False))
    display(
        dfrdf.groupby(['cls','word']).mean().sort_values(
            'prob',ascending=False
        )
    )
    display(dfcdf.groupby(['cls','feat']).mean().sort_values('coeff',ascending=False))


In [24]:
# dfrdf,dfcdf=classify_runs(dfcr,'is_kw',n=100,num_runs=100)
# show_results(dfrdf,dfcdf)

In [25]:
# dfrdf.groupby('word').mean().correct.describe()

In [26]:
# dfrdf2,dfcdf2=classify_runs(dfcr,'cls',n=100,num_runs=100)
# show_results(dfrdf2,dfcdf2)

In [27]:
# for cls,clsdf in dfcdf.groupby('cls'):
#     printm('#### '+cls)
#     display(clsdf.groupby('feat').mean().sort_values('coeff'))

## Classify by vectors

In [28]:
periods=get_periods_bystep(ybin=YBIN_WDIST,ymin=YMIN_WDIST,ymax=YMAX_WDIST)
megavecs = pd.concat((vecs(p) for p in tqdm(periods)), axis=1).dropna()
megavecs.columns=[f'V{i:03}' for i in range(len(megavecs.columns))]
megavecs

100%|██████████| 10/10 [00:02<00:00,  3.88it/s]


,V000,V001,V002,V003,V004,V005,V006,V007,V008,V009,...,V990,V991,V992,V993,V994,V995,V996,V997,V998,V999
the,1.640115,-2.087521,0.213608,0.078114,-0.071124,0.859125,2.268796,-1.487292,-0.525002,-0.457243,...,1.568061,-2.014710,2.500374,-2.539902,1.072781,-2.376514,-2.982754,1.349844,0.439241,1.236372
of,0.190439,-0.866963,0.929591,0.578564,0.213418,0.811845,-0.029139,-0.425810,-0.372906,0.017582,...,0.125025,-2.996547,1.809630,-2.492663,2.434406,-0.284094,-3.579407,1.657086,-0.056593,3.985109
and,-0.062309,-2.198283,-0.220619,0.070905,0.706836,0.860528,-1.325023,-0.873240,0.445097,1.220272,...,2.575425,0.108887,2.398893,-0.145933,1.354571,0.605021,-0.986901,-1.163337,1.923326,0.512059
to,-0.289331,1.067684,-0.878571,0.950415,1.447019,0.288246,0.321682,-1.369376,0.073178,0.437102,...,-2.128720,-0.362169,-0.837858,4.685696,0.598181,-0.862110,0.685494,0.697420,1.870488,-6.970016
in,0.118826,0.526106,0.621032,-0.468411,0.616529,3.164340,1.111433,-0.731922,-1.462903,0.134878,...,-1.925667,-4.113593,-2.149961,-2.513463,-0.737690,-0.119966,-0.436624,-3.059718,-1.288278,2.269088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
crowl,-0.201883,0.131957,-0.095798,0.225772,-0.094674,0.009979,0.091402,0.265761,-0.003123,0.103749,...,1.306091,1.095505,-0.127372,0.820017,0.197384,0.309662,0.206176,-0.894866,0.100922,-0.167689
capon,0.022948,0.106573,0.111004,-0.020038,-0.016595,-0.057362,-0.073647,0.038136,-0.175765,-0.062160,...,1.215393,0.593674,0.663983,0.841800,-1.343514,0.891975,-0.014438,1.042555,2.174658,1.020797
ilks,-0.180069,0.338676,-0.020959,0.179248,-0.084018,-0.133188,0.125965,0.340620,-0.104229,0.015576,...,1.003641,0.133743,-0.339063,0.680987,1.172100,0.767367,-0.508360,-0.578020,-0.368269,-0.149040
enigma,-0.227806,0.245482,0.039816,0.141813,-0.048178,-0.168835,0.169444,0.247552,-0.108038,-0.034949,...,2.718237,3.340461,-0.845794,-0.451779,2.228901,0.977437,3.357971,2.344290,-4.016999,-4.361169


In [29]:
dfclasses_megavecs = dfclasses[['cls']].join(megavecs).dropna()
dfclasses_megavecs

,cls,V000,V001,V002,V003,V004,V005,V006,V007,V008,...,V990,V991,V992,V993,V994,V995,V996,V997,V998,V999
word,,,,,,,,,,,,,,,,,,,,,
abbe,Pivots_1700-1770,-0.311036,0.131913,0.042995,0.146563,0.014886,-0.507833,0.402457,0.427041,-0.134128,...,2.124931,-2.899967,-1.260145,0.056112,0.825855,3.791447,-0.232210,2.540206,-1.859011,-2.518042
abbey,Pivots_Never,-0.568738,0.639067,-0.175317,0.189669,0.237613,-0.288592,0.374440,0.875437,0.109077,...,2.814333,0.765655,4.992216,-0.066594,-1.037334,4.063489,-1.908823,3.410000,0.691745,-2.695126
abbot,Pivots_1770-1830,0.386938,1.600500,0.493854,0.898948,1.155329,-3.645365,2.926386,2.344925,-0.605005,...,4.843101,0.698751,1.992787,2.113519,-1.471995,1.561545,-2.762937,8.172411,-0.063868,-4.834991
abhorrence,Pivots_Never,-0.793633,-0.191839,-0.082016,0.948121,-0.259492,-0.289842,-0.921409,0.105095,0.459752,...,0.571766,0.275591,-2.271841,-1.209566,-1.655049,-2.291663,-2.966774,-2.193255,0.839184,-0.726268
abilities,Pivots_1770-1830,-1.171257,-0.811124,0.372898,1.415380,-0.216913,0.172731,-1.131631,-0.275246,0.481719,...,2.044370,2.794816,-0.499223,-0.729770,-2.301967,1.866834,4.415841,-2.645139,-3.043596,-6.208253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youth,Pivots_Never,1.422439,-0.730858,0.370712,-0.722246,0.623076,-0.763747,-1.538121,-1.837886,0.893138,...,0.848887,1.485101,-1.034247,1.799521,-2.653159,0.402835,7.204577,-2.516169,2.116840,-3.716177
youthful,Pivots_1770-1830,-0.028644,-0.146982,-0.151027,-0.449337,-0.189431,-0.142284,-0.437363,-0.075363,0.292119,...,2.208073,-0.935456,0.736356,1.364295,-0.460627,0.372110,4.572460,0.322884,1.105881,-2.210541
youths,Pivots_1770-1830,-0.277759,-0.351806,0.081249,-0.567034,0.387303,-0.180465,-0.028841,0.354453,-0.004445,...,1.986868,0.956894,-1.103731,1.871308,1.179462,-0.363547,2.532854,-0.317852,1.529700,-0.569882


In [36]:
dfr_vecs = classify(dfclasses_megavecs, 'cls', 'Pivots_1770-1830')

Running SVC classifier: 100%|██████████| 2/2 [02:57<00:00, 88.59s/it] 

DT Accuracy:0.5125, ROC:0.5124, F1:0.5004
GBDT Accuracy:0.5658, ROC:0.5682, F1:0.5788
KNN Accuracy:0.5433, ROC:0.5423, F1:0.521
LR Accuracy:0.5467, ROC:0.5475, F1:0.5444
RF Accuracy:0.5617, ROC:0.5624, F1:0.558
SVC Accuracy:0.5742, ROC:0.5743, F1:0.5651


In [ ]:
dfrdf_vecs, dfcdf_vecs = classify_runs(dfclasses_megavecs)

In [ ]:
dfrdf_vecs.groupby('cls').mean().sort_values('correct')

## By simchange

In [ ]:
cdistmatdf1,cdistmatdf2,cdistmatdf_ch = get_cdist_change_data(force=False)

In [ ]:
cdistmatdf_ch

In [ ]:
from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(rotation=None, n_factors=3)

In [ ]:
fa.fit(cdistmatdf_ch)